# Example of a workflow building for large sample

When a star has multiple observations, it is useful to make a loop over all of the spectra files rather than running each one individually. To obtain outputs of interest, only one cleaned line mask is necessary since all the spectra files are from the same star. With the cleaned mask and .s files, you must loop over the `run_lsdpy` function to calculate outputs for each inputted spectra file.

In the below tutorial, we will be walking you through how to calculate data from multiple spectra files for the same star. It is recomended you first look through the `OneObservationFlow_Tutorial` for additional background on individual steps. We have provided three spectra files: hd46328_test_1.s, hd46328_test_2.s, and hd46328_test_3.s for $\xi^1$ CMa (HD 46328; [Erba et al. 2021](https://doi.org/10.1093/mnras/stab1454)) and the long list (LongList_T27000G35.dat) from the Vienna Atomic Line Database (VALD; [Ryabchikova et al. 2015](https://ui.adsabs.harvard.edu/abs/2015PhyS...90e4005R/abstract)).

## 0. Importing specpolFlow and other packages

In [ ]:
import numpy as np
import pandas as pd
import astropy.units as u
import astropy.constants as const
import specpolFlow as pol
import LSDpy

import copy
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

## 1. Creating & Cleaning LSD Line Mask

Below, we specify the name of the VALD long list (only one is needed since all observations are for the same star) and the name and location of our created mask. Next, we remove regions 100 km $\text{s}^{-1}$ (specific to this example) around the Balmer series and Balmer gap. 

In [ ]:
# making the mask file from the VALD line list
input_filename = 'OneObservationFlow_tutorialfiles/LongList_T27000G35.dat'
output_filename = 'OneObservationFlow_tutorialfiles/test_output/T27000G35_depth0.02.mask'
mask = pol.make_mask(input_filename, output_filename, depthCutoff = 0.02, atomsOnly = True)

# using default regions for cleaning
velrange = 100
ExcludeRegions = pol.get_Balmer_regions_default(velrange) + pol.get_telluric_regions_default()

# cleaning and saving the mask
clean_mask_filename = 'OneObservationFlow_tutorialfiles/test_output/hd46328_test_depth0.02_clean.mask'
mask.clean(ExcludeRegions).save(clean_mask_filename)

## 2. Create LSD Profile

Here, we create the LSD files for all three spectra file by looping over the file names.

In [ ]:
# creating a template for the filenames
obsfile = 'OneObservationFlow_tutorialfiles/hd46328_test_{}.s'
lsdfile = 'OneObservationFlow_tutorialfiles/test_output/hd46328_test_{}.lsd'

for i in range(0,3):
    lsd, mod = pol.run_lsdpy(obs = obsfile.format(i+1), 
            mask = clean_mask_filename, outName = lsdfile.format(i+1), 
            velStart = - 100.0, velEnd = 100.0, velPixel = 2.6, 
            normDepth = 0.2, normLande = 1.2, normWave = 500.0,
            fLSDPlotImg=0)

We can also output a .pdf with all the LSD plots for each observation which is useful for ensuring that the LSD profiles look as expected. 

Note that `%%capture` suppresses the cell's output.

In [ ]:
%%capture
with PdfPages('OneObservationFlow_tutorialfiles/test_output/hd46328_test.pdf') as pdf:

    file = 'OneObservationFlow_tutorialfiles/hd46328_test_{}.s'
    outfile = 'OneObservationFlow_tutorialfiles/test_output/hd46328_test_{}.lsd'

    for i in range(0,3):

        lsd = pol.read_lsd(outfile.format(i+1))
        fig, ax = lsd.plot()

        fig.suptitle('HD46328 test' + '-'+ str(i+1),y = 0.92)
        pdf.savefig(fig)

## 3. Bz Calculation

We can then repeat the above steps for the Bz calculation. Note that each observation has a different `vrad`. One method to resolve this is to make a `vrad` array and put that inside the loop as done below. 

In [ ]:
# Bz calculation using manual cog selection
star = 'hd46328'
vsini = 20
vrad = [12,42,16]

for i in range(3):
    lsdpath = 'OneObservationFlow_tutorialfiles/test_output/hd46328_test_{}.lsd'
    velrange = [vrad[i]-30,vrad[i]+30]
    bzwidth = 20
    lsd = pol.read_lsd(lsdpath.format(i+1))
    
    Bz, fig = lsd.calc_bz(cog = vrad[i], velrange = velrange, plot = True, bzwidth = bzwidth)

In [ ]:
# Bz calculation using manual cog selection
star = 'hd46328'
vsini = 20
vrad = vrad = [12,42,16]

start = 0
for i in range(3):
    lsdpath = 'OneObservationFlow_tutorialfiles/test_output/hd46328_test_{}.lsd'
    velrange = [vrad[i]-30,vrad[i]+30]
    bzwidth = 20
    lsd = pol.read_lsd(lsdpath.format(i+1))
    Bz, fig = lsd.calc_bz(cog = vrad[i], velrange = velrange, plot = True, bzwidth = bzwidth)
    Bz = pd.DataFrame.from_dict(Bz,'index').T
    Bz.insert(0,"Star", star, True)
    if start == 0:
        Bz_table = copy.copy(Bz)
        start = 1
    else:
        Bz_table = pd.concat([Bz_table, Bz], ignore_index = True)

As before, we can also make a .pdf for all the Bz plots.

In [ ]:
%%capture
start = 0
with PdfPages('OneObservationFlow_tutorialfiles/test_output/hd46328_test_Bz.pdf') as pdf:

    # Bz calculation using manual cog selection
    star = 'hd46328'

    vsini = 20
    vrad = vrad = [12,42,16]

    for i in range(3):
        lsdpath = 'OneObservationFlow_tutorialfiles/test_output/hd46328_test_{}.lsd'
        velrange = [vrad[i]-30,vrad[i]+30]
        bzwidth = 20
        lsd = pol.read_lsd(lsdpath.format(i+1))
        Bz, fig = lsd.calc_bz(cog = vrad[i], velrange = velrange, plot = True, bzwidth = bzwidth)
        
        Bz = pd.DataFrame.from_dict(Bz,'index').T
        Bz.insert(0,"Star", star, True)
        if start == 0:
            Bz_table = copy.copy(Bz)
            start = 1
        else:
            Bz_table = pd.concat([Bz_table, Bz], ignore_index = True)
        fig.suptitle('{} - {}'.format(star,i+1),fontsize = 20,y = 0.92)
            
        pdf.savefig(fig)


Often times we also want to output a single table of all the Bz outputs for each observation rather than a different table for each observation. Below is a single output table with the Bz outputs for every observation in the loop.

In [ ]:
Bz_table